# Water-bridge Interactions

This tutorial showcases how to use ProLIF to generate an interaction fingerprint including water-mediated hydrogen-bond interactions, and analyze the interactions for a ligand-protein complex.

## Preparation for MD trajectory

Let's start by importing MDAnalysis and ProLIF to read our tutorial files and create selections for the ligand, protein and water.

:::{important}
It is advised to select for the protein and water selection only the residues in close distance to the ligand,
else the generation of the fingerprint will be time consuming due to the amount of analyzed atoms.
:::

For the selection of the protein in this tutorial we only select the residues around 12.0 Å of the ligand.

:::{tip}
For the water component it is possible to update the `AtomGroup` distance-based selection at every frame,
which is convenient considering the large movements of water molecules during most simulations.
**Do not use `updating=True` for the protein selection**, it will produce wrong results.
:::

For the water selection we select the water residues around 8 Å of the ligand or protein.
You may adjust this distance threshold based on whether you're investigating higher-order
water-mediated interactions (which require a higher threshold to select all relevant waters)
or not.

In [ ]:
import MDAnalysis as mda
import pandas as pd
import prolif as plf

# load example topology and trajectory
u = mda.Universe(plf.datafiles.WATER_TOP, plf.datafiles.WATER_TRAJ)

# create selections for the ligand and protein
ligand_selection = u.select_atoms("resname QNB")
protein_selection = u.select_atoms(
    "protein and byres around 12 group ligand",
    ligand=ligand_selection,
)
water_selection = u.select_atoms(
    "resname TIP3 and byres around 8 (group ligand or group pocket)",
    ligand=ligand_selection,
    pocket=protein_selection,
    updating=True,
)
ligand_selection, protein_selection, water_selection

## Preparation for docking poses

To run a water-bridge analysis with ProLIF for a PDB file and docking poses, you will need
to create 3 objects: a protein molecule (`plf.Molecule`), a collection of ligand molecules
(e.g. a list of `plf.Molecule`), and a water molecule (`plf.Molecule`).

**To create the protein and ligand(s) for your use case, refer to the {doc}`docking </notebooks/docking>` tutorial.**

:::{warning}
To keep the size of test files to a minimum, here we will create molecules directly
from the MD trajectory above, instead of using PDB/MOL2/SDF/PDBQT files.
Please follow the instructions given in the relevant tutorial closely instead.
:::

In [ ]:
# create list of ligand poses for each frame in the MD trajectory
ligand_poses = [plf.Molecule.from_mda(ligand_selection) for ts in u.trajectory]
# create protein
protein_mol = plf.Molecule.from_mda(protein_selection + water_selection)

With this out of the way, we can now create the molecule object containing our waters.

TODO: function for separating the protein from waters

In [ ]:
# TODO
protein_mol = plf.Molecule.from_mda(protein_selection)
water_mol = plf.Molecule.from_mda(water_selection)

## Analysis

Now we perform the calculation of the interactions.
In this tutorial we only focus on the water bridge interactions, but you can also include the other typical ligand-protein interactions in the list of interactions.

:::{important}
When using `WaterBridge`, you must specify the `water` parameter with either your atomgroup selection
if using an MD trajectory, or the water molecule if using PDB/docking files.
:::

By default, the `WaterBridge` interaction will only look at bridges including a single water molecule, i.e. `ligand---water---protein`.
Here, we will look at the water bridges up to order 3 (i.e. there can be up to 3 water molecules),
for this we explicitely provide `order=3` in the parameters of the `WaterBridge` interaction (defaults to `1`).

In [ ]:
# for docking poses, replace water_selection with water_mol
fp = plf.Fingerprint(
    ["WaterBridge"], parameters={"WaterBridge": {"water": water_selection, "order": 3}}
)

Then we simply run the analysis. The way the `WaterBridge` analysis is performed under the hood,
three successive runs are executed:
- between the ligand and water
- between the protein and water
- between the water molecules

The results are then collated together using `networkx`.

:::{note}
For practical reasons, the `WaterBridge` analysis only runs in serial.
:::

In [ ]:
# for MD trajectories
fp.run(u.trajectory, ligand_selection, protein_selection)

```python
# for docking poses
fp.run_from_iterable(ligand_poses, protein_mol)
```

## Visualization

The example files that we use consists of 20 frames/poses.
Let's analyze the water bridge interactions that are present in the trajectory.
For this we generate a DataFrame, which shows which interactions occur during each frame of the simulation.

In [ ]:
df = fp.to_dataframe()
df

We now sort the values to identify which of the water bridge interactions appears more frequently than the others.

In [ ]:
# percentage of the trajectory where each interaction is present
df.mean().sort_values(ascending=False).to_frame(name="%").T * 100

We can also analyze the water bridge interactions using a barcode plot.

In [ ]:
fp.plot_barcode(figsize=(8, 3))

We can also visualize water-mediated interactions using the `LigNetwork` plot.
Here we can also see the water bridges with higher orders, where the water molecules interact 
with each other thus building water bridges.

:::{tip}
The threshold for interactions to be displayed in `fp.plot_lignetwork()` is `0.3`. Thus only interactions with an occurence of more than 30 % will appear with the default settings, so don't forget to adjust the threshold if you want to see interactions with lower occurence.
:::

In [ ]:
ligand_mol = plf.Molecule.from_mda(ligand_selection)
view = fp.plot_lignetwork(ligand_mol, threshold=0.05)
view

We can also visualize water-mediated interactions in 3D with the `plot_3d` function. Here is an example of the water-mediated interaction between the protein and ligand present in frame/pose `0`.

:::{note}
For `plot_3d` we need to provide the `protein`, `ligand` and `water` objects as `plf.Molecule`,
thus if you're using an MD trajectory, a conversion from the `mda.AtomGroup` selection
which we used for `fp.run()` may be required:
:::

In [ ]:
# preparation for MD trajectories only
frame = 2
u.trajectory[frame]  # seek frame to update coordinates of atomgroup objects
ligand_mol = plf.Molecule.from_mda(ligand_selection, use_segid=fp.use_segid)
protein_mol = plf.Molecule.from_mda(protein_selection, use_segid=fp.use_segid)
water_mol = plf.Molecule.from_mda(water_selection, use_segid=fp.use_segid)

In [ ]:
frame = 2
view = fp.plot_3d(ligand_mol, protein_mol, water_mol, frame=frame)
view

You can also compare different poses/frames, please refer to the relevant {ref}`source/tutorials:Tutorials` and
simply add `water_mol` in the `plot_3d` call.

## Water Bridge Interaction Metadata

The current example showed if specific water bridge interactions are present or not during the simulation.
During the analysis, some metadata about the interaction is stored:
- the indices of atoms involved in each component (ligan, protein or water),
- the "order" of the water-mediated interaction, i.e. how many water molecules are involved in the bridge,
- the residue identifier of the water molecules,
- the role of the ligand and protein (H-bond acceptor or donor),
- distances for each HBond interaction forming the bridge (and their sum).

In [ ]:
frame = 0
all_interaction_data = fp.ifp[frame].interactions()
next(all_interaction_data)

Next we show how to access and process the metadata stored after the calculation of the interactions, using a pandas Dataframe for convenience.

In [ ]:
all_metadata = []

for frame, ifp in fp.ifp.items():
    for interaction_data in ifp.interactions():
        if interaction_data.interaction == "WaterBridge":
            flat = {
                "frame": frame,
                "ligand_residue": interaction_data.ligand,
                "protein_residue": interaction_data.protein,
                "water_residues": " ".join(
                    map(str, interaction_data.metadata["water_residues"])
                ),
                "order": interaction_data.metadata["order"],
                "ligand_role": interaction_data.metadata["ligand_role"],
                "protein_role": interaction_data.metadata["protein_role"],
                "total_distance": interaction_data.metadata["distance"],
            }
            all_metadata.append(flat)

df_metadata = pd.DataFrame(all_metadata)
df_metadata

We can now use this information to access interactions of orders 2 and 3 only to perform further analysis.

In [ ]:
# count the occurence of each water residue in bridged interactions of higher order
(
    df_metadata[df_metadata["order"].isin([2, 3])]["water_residues"]
    .str.split(" ")
    .explode()
    .value_counts()
)